## Framework

- Data cleaning and formatting 
- Exploratory data analysis
- Feature engineering and selection
- Compare several machine learning models on a performance metric
- Perform hyper-parameter tuning on the best model 
- Evaluate the best model on the testing set
- Interpret the model results
- Draw conclusions and document work

In [733]:
#load packages

#lm pacakges
from sklearn import tree 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.preprocessing import normalize, scale, Normalizer, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.dummy import DummyClassifier

#other 
import numpy as np
import pandas as pd
import pickle 
import graphviz
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling
pd.set_option("display.max_colwidth", 200)

import altair as alt
import time

import autotime

In [734]:
#ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

In [855]:
df1 = pd.read_csv('data/01_oscar_data.csv')
df2 = pd.read_csv('data/02_oscar_data.csv')

In [856]:
#df1.profile_report(style={'full_width':True})

- remove all the categorical for wins and nominations so model only is concern with # of nominations and wins 
    - might have to return to this later

In [857]:
df1.head(1)

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,gross,release_date,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Oscar_nominated_categories,Golden_Globes_won,Golden_Globes_won_categories,Golden_Globes_nominated,Golden_Globes_nominated_categories,BAFTA_won,BAFTA_won_categories,BAFTA_nominated,BAFTA_nominated_categories,Screen_Actors_Guild_won,Screen_Actors_Guild_won_categories,Screen_Actors_Guild_nominated,Screen_Actors_Guild_nominated_categories,Critics_Choice_won,Critics_Choice_won_categories,Critics_Choice_nominated,Critics_Choice_nominated_categories,Directors_Guild_won,Directors_Guild_won_categories,Directors_Guild_nominated,Directors_Guild_nominated_categories,Producers_Guild_won,Producers_Guild_won_categories,Producers_Guild_nominated,Producers_Guild_nominated_categories,Art_Directors_Guild_won,Art_Directors_Guild_won_categories,Art_Directors_Guild_nominated,Art_Directors_Guild_nominated_categories,Writers_Guild_won,Writers_Guild_won_categories,Writers_Guild_nominated,Writers_Guild_nominated_categories,Costume_Designers_Guild_won,Costume_Designers_Guild_won_categories,Costume_Designers_Guild_nominated,Costume_Designers_Guild_nominated_categories,Online_Film_Television_Association_won,Online_Film_Television_Association_won_categories,Online_Film_Television_Association_nominated,Online_Film_Television_Association_nominated_categories,Online_Film_Critics_Society_won,Online_Film_Critics_Society_won_categories,Online_Film_Critics_Society_nominated,Online_Film_Critics_Society_nominated_categories,People_Choice_won,People_Choice_won_categories,People_Choice_nominated,People_Choice_nominated_categories,London_Critics_Circle_Film_won,London_Critics_Circle_Film_won_categories,London_Critics_Circle_Film_nominated,London_Critics_Circle_Film_nominated_categories,American_Cinema_Editors_won,American_Cinema_Editors_won_categories,American_Cinema_Editors_nominated,American_Cinema_Editors_nominated_categories,Hollywood_Film_won,Hollywood_Film_won_categories,Hollywood_Film_nominated,Hollywood_Film_nominated_categories,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_won_categories,Austin_Film_Critics_Association_nominated,Austin_Film_Critics_Association_nominated_categories,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_won_categories,Denver_Film_Critics_Society_nominated,Denver_Film_Critics_Society_nominated_categories,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_won_categories,Boston_Society_of_Film_Critics_nominated,Boston_Society_of_Film_Critics_nominated_categories,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_won_categories,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
0,2001,Kate & Leopold,tt0035423,PG-13,118,Comedy|Fantasy|Romance,6.4,44.0,An English Duke from 1876 is inadvertedly dragged to modern day New York where he falls for a plucky advertising executive.,66660,47100000.0,2001-12-25,318.0,125.0,2363.0,1,4,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1,"Best Music, Original Song",1,Best Original Song - Motion Picture,2,Best Original Song - Motion Picture|Best Performance by an Actor in a Motion Picture - Comedy

# Data cleaning and formatting and Exploratory data analysis

In [858]:
#added back in golden globes for test
df1_temp = df1.drop(['movie_id', 'release_date', 'synopsis', 'release_date.year','release_date.month', 'release_date.day-of-month', 'release_date.day-of-week', 'American_Cinema_Editors_nominated_categories', 'Art_Directors_Guild_nominated_categories', 'Austin_Film_Critics_Association_nominated_categories', 'BAFTA_nominated_categories', 'Boston_Society_of_Film_Critics_nominated_categories', 'Costume_Designers_Guild_nominated_categories', 'Costume_Designers_Guild_won_categories', 'Critics_Choice_nominated_categories', 'Denver_Film_Critics_Society_nominated_categories', 'Directors_Guild_nominated_categories','Golden_Globes_nominated_categories','Hollywood_Film_nominated_categories', 'London_Critics_Circle_Film_nominated_categories', 'Los_Angeles_Film_Critics_Association_nominated_categories', 'New_York_Film_Critics_Circle_nominated_categories', 'Online_Film_Critics_Society_nominated_categories', 'Online_Film_Television_Association_nominated_categories', 'People_Choice_nominated_categories',  'Producers_Guild_nominated_categories', 'Screen_Actors_Guild_nominated_categories', 'Writers_Guild_nominated_categories', 'American_Cinema_Editors_won_categories', 'London_Critics_Circle_Film_won_categories'], axis = 1)

In [859]:
df1.head(0)

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,gross,release_date,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Oscar_nominated_categories,Golden_Globes_won,Golden_Globes_won_categories,Golden_Globes_nominated,Golden_Globes_nominated_categories,BAFTA_won,BAFTA_won_categories,BAFTA_nominated,BAFTA_nominated_categories,Screen_Actors_Guild_won,Screen_Actors_Guild_won_categories,Screen_Actors_Guild_nominated,Screen_Actors_Guild_nominated_categories,Critics_Choice_won,Critics_Choice_won_categories,Critics_Choice_nominated,Critics_Choice_nominated_categories,Directors_Guild_won,Directors_Guild_won_categories,Directors_Guild_nominated,Directors_Guild_nominated_categories,Producers_Guild_won,Producers_Guild_won_categories,Producers_Guild_nominated,Producers_Guild_nominated_categories,Art_Directors_Guild_won,Art_Directors_Guild_won_categories,Art_Directors_Guild_nominated,Art_Directors_Guild_nominated_categories,Writers_Guild_won,Writers_Guild_won_categories,Writers_Guild_nominated,Writers_Guild_nominated_categories,Costume_Designers_Guild_won,Costume_Designers_Guild_won_categories,Costume_Designers_Guild_nominated,Costume_Designers_Guild_nominated_categories,Online_Film_Television_Association_won,Online_Film_Television_Association_won_categories,Online_Film_Television_Association_nominated,Online_Film_Television_Association_nominated_categories,Online_Film_Critics_Society_won,Online_Film_Critics_Society_won_categories,Online_Film_Critics_Society_nominated,Online_Film_Critics_Society_nominated_categories,People_Choice_won,People_Choice_won_categories,People_Choice_nominated,People_Choice_nominated_categories,London_Critics_Circle_Film_won,London_Critics_Circle_Film_won_categories,London_Critics_Circle_Film_nominated,London_Critics_Circle_Film_nominated_categories,American_Cinema_Editors_won,American_Cinema_Editors_won_categories,American_Cinema_Editors_nominated,American_Cinema_Editors_nominated_categories,Hollywood_Film_won,Hollywood_Film_won_categories,Hollywood_Film_nominated,Hollywood_Film_nominated_categories,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_won_categories,Austin_Film_Critics_Association_nominated,Austin_Film_Critics_Association_nominated_categories,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_won_categories,Denver_Film_Critics_Society_nominated,Denver_Film_Critics_Society_nominated_categories,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_won_categories,Boston_Society_of_Film_Critics_nominated,Boston_Society_of_Film_Critics_nominated_categories,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_won_categories,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week


In [860]:
factor_features = ['Art_Directors_Guild_won_categories', 'Austin_Film_Critics_Association_won_categories', 'BAFTA_won_categories', 'Boston_Society_of_Film_Critics_won_categories', 'Critics_Choice_won_categories', 'Denver_Film_Critics_Society_won_categories', 'Directors_Guild_won_categories', 'Golden_Globes_won_categories', 'Hollywood_Film_won_categories', 'London_Critics_Circle_Film_won_categories', 'Los_Angeles_Film_Critics_Association_won_categories', 'New_York_Film_Critics_Circle_won_categories', 'Online_Film_Critics_Society_won_categories', 'Online_Film_Television_Association_won_categories', 'Oscar_nominated_categories','People_Choice_won_categories','Screen_Actors_Guild_won_categories','Writers_Guild_won_categories', 'Producers_Guild_won_categories', ]
awards = ['Art Directors Guild', 'Austin Film Critics Association', 'BAFTA', 'Boston Society of Film Critics', 'Critics Choice', 'Denver Film Critics Society', 'Directors Guild', 'Golden Globes', 'Hollywood Film', 'London Critics Circle Film', 'Los Angeles Film Critics Association', 'New York Film Critics Circle', 'Online Film Critics Society', 'Online Film Television Association', 'Oscar nominated','People Choice','Screen Actors Guild','Writers Guild', 'Producers Guild']



In [861]:
#df1_temp.profile_report(style={'full_width':True})

- Remove Hollywood_Film_won and Hollywood_Film_nominated do to high correlation 
- Can keep Online_Film_Television_Association_nominated and awards_nominations because it does not look like they are related
- give gross and popularity missing values the mediumn value
- remove the missing metascore, certificate, and release_date.month values 
- Deal with the categorical nature of genre using MultiLabelBinarizer

In [862]:
df1_temp = df1_temp.drop(['Hollywood_Film_won', 'Hollywood_Film_nominated'], axis = 1)

In [863]:
df1_temp = df1_temp.dropna(subset=['metascore', 'certificate'])

In [864]:
df1_temp['gross'].fillna((df1_temp['gross'].median()), inplace=True)
df1_temp['popularity'].fillna((df1_temp['popularity'].median()), inplace=True)

### Factor features 

In [865]:
#this one was done before the other factor features and will be left alone. 
df1_temp['genre'] = [x.split('|') for x in df1_temp['genre']]

In [866]:
mlb = MultiLabelBinarizer()
df1_temp = df1_temp.join(pd.DataFrame(mlb.fit_transform(df1_temp.pop('genre')),
                          columns=mlb.classes_,
                          index=df1_temp.index))

In [867]:
factor_features = ['Art_Directors_Guild_won_categories', 'Austin_Film_Critics_Association_won_categories', 'BAFTA_won_categories', 'Boston_Society_of_Film_Critics_won_categories', 'Critics_Choice_won_categories', 'Denver_Film_Critics_Society_won_categories', 'Directors_Guild_won_categories', 'Golden_Globes_won_categories', 'Hollywood_Film_won_categories', 'Los_Angeles_Film_Critics_Association_won_categories', 'New_York_Film_Critics_Circle_won_categories', 'Online_Film_Critics_Society_won_categories', 'Online_Film_Television_Association_won_categories', 'Oscar_nominated_categories','People_Choice_won_categories','Screen_Actors_Guild_won_categories','Writers_Guild_won_categories', 'Producers_Guild_won_categories']
awards = ['Art Directors Guild', 'Austin Film Critics Association', 'BAFTA', 'Boston Society of Film Critics', 'Critics Choice', 'Denver Film Critics Society', 'Directors Guild', 'Golden Globes', 'Hollywood Film', 'Los Angeles Film Critics Association', 'New York Film Critics Circle', 'Online Film Critics Society', 'Online Film Television Association', 'Oscar nominated','People Choice','Screen Actors Guild','Writers Guild', 'Producers Guild']
#'American_Cinema_Editors_won_categories', 'London_Critics_Circle_Film_won_categories',
#'American Cinema Editors', 'London Critics Circle Film',

In [868]:
# factor_features = ['Golden_Globes_won_categories', 'BAFTA_won_categories', 'Critics_Choice_won_categories']
# awards = ['Golden Globes', 'BAFTA', 'Critics Choice']
mlb = MultiLabelBinarizer()

for feature, award in zip(factor_features, awards):
    df1_temp[feature] = df1_temp[feature].astype(str)
    df1_temp[feature] = [x.replace('Best', f'{award} Best') for x in df1_temp[feature]]
    df1_temp[feature] = [x.split('|') for x in df1_temp[feature]]
    df1_temp = df1_temp.join(pd.DataFrame(mlb.fit_transform(df1_temp.pop(feature)),
                             columns=mlb.classes_,
                             index=df1_temp.index))
    df1_temp = df1_temp.drop(['nan'], axis = 1)
    df1_temp[''] = 1
    df1_temp = df1_temp.drop([''], axis = 1)
    df1_temp['Animation'] = 1
    df1_temp = df1_temp.drop(['Animation'], axis = 1)
    

In [869]:
df1_temp.head(0)

,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Histor,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Anthony Asquith Award for Film Music,BAFTA Best Achievement in Special Visual Effects,BAFTA Best Actor in a Leading Role,BAFTA Best Actor in a Supporting Role,BAFTA Best Actress in a Supporting Role,BAFTA Best Adapted Screenplay,BAFTA Best Animated Feature Film,BAFTA Best Animated Featured Film,BAFTA Best Animated Film,BAFTA Best British Film,BAFTA Best Cinematography,BAFTA Best Costume Design,BAFTA Best Director,BAFTA Best Documentary,BAFTA Best Editing,BAFTA Best Feature Film,BAFTA Best Film,BAFTA Best Film Not in the English Language,BAFTA Best Film not in the English Language,BAFTA Best Leading Actor,BAFTA Best Leading Actress,BAFTA Best Make Up & Hair,BAFTA Best Make Up/Hair,BAFTA Best Makeup and Hair,BAFTA Best Original Music,BAFTA Best Original Screenplay,BAFTA Best Performance by an Actor in a Leading Role,BAFTA Best Performance by an Actor in a Supporting Role,BAFTA Best Performance by an Actress in a Leading Role,BAFTA Best Performance by an Actress in a Supporting Role,BAFTA Best Productio

In [870]:
# df1_temp['Golden_Globes_won_categories'] = [x.split('|') for x in df1_temp['Golden_Globes_won_categories']]


In [871]:
# df1_temp.profile_report(style={'full_width':True})

- Fix the histor category history 

In [872]:
df1_temp = df1_temp.reset_index()

In [873]:
df1_temp.query("Histor == 1")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Histor,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Anthony Asquith Award for Film Music,BAFTA Best Achievement in Special Visual Effects,BAFTA Best Actor in a Leading Role,BAFTA Best Actor in a Supporting Role,BAFTA Best Actress in a Supporting Role,BAFTA Best Adapted Screenplay,BAFTA Best Animated Feature Film,BAFTA Best Animated Featured Film,BAFTA Best Animated Film,BAFTA Best British Film,BAFTA Best Cinematography,BAFTA Best Costume Design,BAFTA Best Director,BAFTA Best Documentary,BAFTA Best Editing,BAFTA Best Feature Film,BAFTA Best Film,BAFTA Best Film Not in the English Language,BAFTA Best Film not in the English Language,BAFTA Best Leading Actor,BAFTA Best Leading Actress,BAFTA Best Make Up & Hair,BAFTA Best Make Up/Hair,BAFTA Best Makeup and Hair,BAFTA Best Original Music,BAFTA Best Original Screenplay,BAFTA Best Performance by an Actor in a Leading Role,BAFTA Best Performance by an Actor in a Supporting Role,BAFTA Best Performance by an Actress in a Leading Role,BAFTA Best Performance by an Actress in a Supporting Role,BAFTA Best Pro

In [874]:
df1_temp.iloc[682]["History"] = 1

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [875]:
df1_temp = df1_temp.drop(["Histor", "index"], axis=1) 

- fix the certified column to move tv-ma to r and unrated to not rated

In [876]:
df1_temp = df1_temp.reset_index()

In [877]:
df1_temp.query("certificate == 'TV-MA'")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Anthony Asquith Award for Film Music,BAFTA Best Achievement in Special Visual Effects,BAFTA Best Actor in a Leading Role,BAFTA Best Actor in a Supporting Role,BAFTA Best Actress in a Supporting Role,BAFTA Best Adapted Screenplay,BAFTA Best Animated Feature Film,BAFTA Best Animated Featured Film,BAFTA Best Animated Film,BAFTA Best British Film,BAFTA Best Cinematography,BAFTA Best Costume Design,BAFTA Best Director,BAFTA Best Documentary,BAFTA Best Editing,BAFTA Best Feature Film,BAFTA Best Film,BAFTA Best Film Not in the English Language,BAFTA Best Film not in the English Language,BAFTA Best Leading Actor,BAFTA Best Leading Actress,BAFTA Best Make Up & Hair,BAFTA Best Make Up/Hair,BAFTA Best Makeup and Hair,BAFTA Best Original Music,BAFTA Best Original Screenplay,BAFTA Best Performance by an Actor in a Leading Role,BAFTA Best Performance by an Actor in a Supporting Role,BAFTA Best Performance by an Actress in a Leading Role,BAFTA Best Performance by an Actress in a Supporting Role,BAFTA Best Production

In [878]:
df1_temp.loc[df1_temp['certificate'].str.contains('TV-MA', case=False), 'certificate'] = 'R'

In [879]:
df1_temp.iloc[1159]['certificate'] 

'R'

In [880]:
df1_temp.loc[df1_temp['certificate'].str.contains('Unrated', case=False), 'certificate'] = 'Not Rated'

In [881]:
df1_temp.query("certificate == 'Unrated'")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Anthony Asquith Award for Film Music,BAFTA Best Achievement in Special Visual Effects,BAFTA Best Actor in a Leading Role,BAFTA Best Actor in a Supporting Role,BAFTA Best Actress in a Supporting Role,BAFTA Best Adapted Screenplay,BAFTA Best Animated Feature Film,BAFTA Best Animated Featured Film,BAFTA Best Animated Film,BAFTA Best British Film,BAFTA Best Cinematography,BAFTA Best Costume Design,BAFTA Best Director,BAFTA Best Documentary,BAFTA Best Editing,BAFTA Best Feature Film,BAFTA Best Film,BAFTA Best Film Not in the English Language,BAFTA Best Film not in the English Language,BAFTA Best Leading Actor,BAFTA Best Leading Actress,BAFTA Best Make Up & Hair,BAFTA Best Make Up/Hair,BAFTA Best Makeup and Hair,BAFTA Best Original Music,BAFTA Best Original Screenplay,BAFTA Best Performance by an Actor in a Leading Role,BAFTA Best Performance by an Actor in a Supporting Role,BAFTA Best Performance by an Actress in a Leading Role,BAFTA Best Performance by an Actress in a Supporting Role,BAFTA Best Production

In [882]:
df1_temp = df1_temp.dropna()

In [883]:
df_clean = df1_temp

In [884]:
df_clean

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Anthony Asquith Award for Film Music,BAFTA Best Achievement in Special Visual Effects,BAFTA Best Actor in a Leading Role,BAFTA Best Actor in a Supporting Role,BAFTA Best Actress in a Supporting Role,BAFTA Best Adapted Screenplay,BAFTA Best Animated Feature Film,BAFTA Best Animated Featured Film,BAFTA Best Animated Film,BAFTA Best British Film,BAFTA Best Cinematography,BAFTA Best Costume Design,BAFTA Best Director,BAFTA Best Documentary,BAFTA Best Editing,BAFTA Best Feature Film,BAFTA Best Film,BAFTA Best Film Not in the English Language,BAFTA Best Film not in the English Language,BAFTA Best Leading Actor,BAFTA Best Leading Actress,BAFTA Best Make Up & Hair,BAFTA Best Make Up/Hair,BAFTA Best Makeup and Hair,BAFTA Best Original Music,BAFTA Best Original Screenplay,BAFTA Best Performance by an Actor in a Leading Role,BAFTA Best Performance by an Actor in a Supporting Role,BAFTA Best Performance by an Actress in a Leading Role,BAFTA Best Performance by an Actress in a Supporting Role,BAFTA Best Production

In [885]:
# #structure features 
# df_clean = df_clean.drop(['index'], axis = 1)
# df_clean.head(1)

### Pull the winners of 2018 for back testing 

In [886]:
best_picture_2018_identifiable = df_clean.query("Oscar_Best_Picture_nominated == 'Yes' & year == 2018")
best_director_2018_identifiable = df_clean.query("Oscar_Best_Director_nominated == 'Yes' & year == 2018")
best_actor_2018_identifiable = df_clean.query("Oscar_Best_Actor_nominated == 'Yes' & year == 2018")
best_actress_2018_identifiable = df_clean.query("Oscar_Best_Actress_nominated == 'Yes' & year == 2018")
best_supporting_actor_2018_identifiable = df_clean.query("Oscar_Best_Supporting_Actor_nominated == 'Yes' & year == 2018")
best_supporting_actress_2018_identifiable = df_clean.query("Oscar_Best_Supporting_Actress_nominated == 'Yes' & year == 2018")
best_oriscreen_2018_identifiable = df_clean.query("Oscar_Best_OriScreen_nominated == 'Yes' & year == 2018")
best_adascreen_2018_identifiable = df_clean.query("Oscar_Best_AdaScreen_nominated == 'Yes' & year == 2018")


In [887]:
t = pd.concat([best_actor_2018_identifiable, best_actress_2018_identifiable, best_adascreen_2018_identifiable, best_director_2018_identifiable, best_oriscreen_2018_identifiable, best_picture_2018_identifiable, best_supporting_actor_2018_identifiable, best_supporting_actress_2018_identifiable], axis=0)
best_back_test_2018_identifiable = t.drop_duplicates(keep='last')

df_ml = df_clean.drop(list(best_back_test_2018_identifiable.index))


In [888]:
#final drop of identifable information
df_ml = df_ml.drop(['year', 'movie', 'index'], axis = 1)
best_back_test_2018_unidentifiable = best_back_test_2018_identifiable.drop(['year', 'movie', 'index'], axis = 1)

In [952]:
df_ml

,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Anthony Asquith Award for Film Music,BAFTA Best Achievement in Special Visual Effects,BAFTA Best Actor in a Leading Role,BAFTA Best Actor in a Supporting Role,BAFTA Best Actress in a Supporting Role,BAFTA Best Adapted Screenplay,BAFTA Best Animated Feature Film,BAFTA Best Animated Featured Film,BAFTA Best Animated Film,BAFTA Best British Film,BAFTA Best Cinematography,BAFTA Best Costume Design,BAFTA Best Director,BAFTA Best Documentary,BAFTA Best Editing,BAFTA Best Feature Film,BAFTA Best Film,BAFTA Best Film Not in the English Language,BAFTA Best Film not in the English Language,BAFTA Best Leading Actor,BAFTA Best Leading Actress,BAFTA Best Make Up & Hair,BAFTA Best Make Up/Hair,BAFTA Best Makeup and Hair,BAFTA Best Original Music,BAFTA Best Original Screenplay,BAFTA Best Performance by an Actor in a Leading Role,BAFTA Best Performance by an Actor in a Supporting Role,BAFTA Best Performance by an Actress in a Leading Role,BAFTA Best Performance by an Actress in a Supporting Role,BAFTA Best Production Design,BAFTA Bes

### Create the X and Ys

In [889]:
X = df_ml.drop(['Oscar_Best_Actor_won', 'Oscar_Best_Actress_won', 'Oscar_Best_AdaScreen_won', 'Oscar_Best_Director_won', 'Oscar_Best_OriScreen_won', 'Oscar_Best_Picture_won', "Oscar_Best_Supporting_Actor_won", "Oscar_Best_Supporting_Actress_won"], axis = 1)
y_best_actor = df_ml['Oscar_Best_Actor_won']
y_best_actress = df_ml['Oscar_Best_Actress_won']
y_best_ada_screen = df_ml['Oscar_Best_AdaScreen_won']
y_best_director = df_ml['Oscar_Best_Director_won']
y_best_ori_screen = df_ml['Oscar_Best_OriScreen_won']
y_best_picture = df_ml['Oscar_Best_Picture_won']
y_best_supporting_actor = df_ml['Oscar_Best_Supporting_Actor_won']
y_best_supporting_actress = df_ml['Oscar_Best_Supporting_Actress_won']

### Split X and Y 

In [921]:
#best actor
X_train_best_actor, X_test_best_actor, y_train_best_actor, y_test_best_actor = train_test_split(X,
                                                                          y_best_actor,
                                                                          test_size=0.2)
#best actoress 
X_train_best_actress, X_test_best_actress, y_train_best_actress, y_test_best_actress = train_test_split(X,
                                                                          y_best_actress,
                                                                          test_size=0.2)

#best adaptive screen play 
X_train_best_ada_screen, X_test_best_ada_screen, y_train_best_ada_screen, y_test_best_ada_screen = train_test_split(X,
                                                                          y_best_ada_screen,
                                                                          test_size=0.2)
#best director
X_train_best_director, X_test_best_director, y_train_best_director, y_test_best_director = train_test_split(X,
                                                                          y_best_director,
                                                                          test_size=0.2)

#best orginal screen play 
X_train_best_ori_screen, X_test_best_ori_screen, y_train_best_ori_screen, y_test_best_ori_screen = train_test_split(X,
                                                                          y_best_ori_screen,
                                                                          test_size=0.2)

#best picture 
X_train_best_picture, X_test_best_picture, y_train_best_picture, y_test_best_picture = train_test_split(X,
                                                                          y_best_picture,
                                                                          test_size=0.2)

#best supporting actor 
X_train_best_supporting_actor, X_test_best_supporting_actor, y_train_best_supporting_actor, y_test_best_supporting_actor = train_test_split(X,
                                                                          y_best_supporting_actor,
                                                                          test_size=0.2)

#best suporting actress 
X_train_best_supporting_actress, X_test_best_supporting_actress, y_train_best_supporting_actress, y_test_best_supporting_actress = train_test_split(X,
                                                                          y_best_supporting_actress,
                                                                          test_size=0.2)


In [922]:
#split 
numeric_features = ['duration', 'rate', 'metascore', 'gross', 'user_reviews', 'critic_reviews', 'popularity']
    
    
categorical_features = ['certificate', 'Oscar_Best_Picture_nominated', 'Oscar_Best_Director_nominated', 'Oscar_Best_Actor_nominated', 'Oscar_Best_Actress_nominated', 'Oscar_Best_Supporting_Actress_nominated', 'Oscar_Best_OriScreen_nominated']

### Processing Pipeline 

In [923]:
# transform the data to standardize the values in the data 
preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), numeric_features),
        ('ohe', OneHotEncoder(drop="first"), categorical_features)])


In [924]:
def get_scores(model, 
                X_train, y_train,
                X_test, y_test, 
                show = True
               ):
    """
    Returns train and validation error given a model
    train and validation X and y portions
    Parameters
    ----------
    model: sklearn classifier model
        The sklearn model
    X_train: numpy.ndarray        
        The X part of the train set
    y_train: numpy.ndarray
        The y part of the train set    
    X_valid: numpy.ndarray        
        The X part of the validation set
    y_valid: numpy.ndarray
        The y part of the validation set    
    Returns
    -------
        train_err: float
        test_err: float
            
    """ 
    
    if show: 
        print("Training error:   %.2f" % (1-model.score(X_train, y_train)))
        print("Validation error: %.2f" % (1-model.score(X_test, y_test)))
        print('\n')
    return (1-model.score(X_train, y_train)), (1-model.score(X_test, y_test))

In [925]:
def diff_class_ml(X_train, X_test, y_train, y_test):
    """
    Returns train error, validation error and time given an ensemble of models 
    Parameters
    ----------
    X_train: numpy.ndarray        
        The X part of the train set
    y_train: numpy.ndarray
        The y part of the train set    
    X_valid: numpy.ndarray        
        The X part of the validation set
    y_valid: numpy.ndarray
        The y part of the validation set    
    Returns
    -------
        pd.DataFrame
            
    """ 
    # Lets create an empty dictionary to store all the results
    results_dict = {}
    
    models = {
          'dummy': DummyClassifier(), 
          'decision tree': DecisionTreeClassifier(),
          'kNN': KNeighborsClassifier(),
          'logistic regression': LogisticRegression(),
          'random forest' : RandomForestClassifier(), 
          'xgboost' : XGBClassifier(),
          'lgbm': LGBMClassifier()
         }

    for model_name, model in models.items():
        t = time.time()
        #print(model_name, ":")    
        clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', model)])
        clf.fit(X_train, y_train);
        tr_err, valid_err = get_scores(clf, X_train, y_train, 
                                       X_test, y_test, show = False)
        elapsed_time = time.time() - t
        results_dict[model_name] = [round(tr_err,3), round(valid_err,3), round(elapsed_time,4)]
        #print("Elapsed time: %.1f s" % elapsed_time)
    
    results_df = pd.DataFrame(results_dict).T
    results_df.columns = ["Train error", "Validation error", "Time in seconds"]
    return results_df

In [953]:
def oscar_predict(X_train, y_train, dataframe, model):
    """
    Shows Classifier and Probability for the Oscar nomination (or other movie)
    
    Parameters
    ----------
    X_train: numpy.ndarray 
        The X training set of the selected catagory 
    y_train: numpy.ndarray
        The y training set the selected catagory 
    dataframe: numpy.ndarray
        The movies that need to be predicted if they won their category 
    model: sklearn classifier model
        The sklearn model such as LogisticRegression()
    
    Returns
    -------
        pd.DataFrame
    
    """
    # Lets create an empty dictionary to store all the results
    results_dict = {}
    
    movies = list(dataframe.index)
    movies_names = list(dataframe["movie"])
    dataframe_X = dataframe.drop(['year', 'movie', 'index', 'Oscar_Best_Actor_won', 'Oscar_Best_Actress_won', 'Oscar_Best_AdaScreen_won', 'Oscar_Best_Director_won', 'Oscar_Best_OriScreen_won', 'Oscar_Best_Picture_won', "Oscar_Best_Supporting_Actor_won", "Oscar_Best_Supporting_Actress_won"], axis = 1)

    clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', model)])
    model = clf.fit(X_train, y_train);
        
    for movie, movie_name in zip(movies, movies_names):
        movie_predict = dataframe_X.loc[[movie]]
        classifier = model.predict(movie_predict)
        classifier = classifier[0]
        prob = model.predict_proba(movie_predict)
        prob = prob[0][1]
        results_dict[movie_name] = [classifier, round(prob,8)]
    
    results_df = pd.DataFrame(results_dict).T
    results_df.columns = ["Classifier", "Probability of it Winning"]
    return results_df

In [954]:
def normalized_winner(predict_df):
    """
    Normalized the chance of a movie winning an oscar
    
    Parameters
    ----------
    predict_df: pd.DataFram from oscar_predict()
        
    Returns
    -------
        pd.DataFrame
    
    """
    predict_df = predict_df.drop(["Classifier"], axis = 1)
    predict_df["Chance of Winning"] = predict_df["Probability of it Winning"]/predict_df["Probability of it Winning"].sum()
    predict_df = predict_df.drop(["Probability of it Winning"], axis = 1)
    predict_df ["Chance of Winning"] = pd.Series(["{0:.2f}%".format(val * 100) for val in predict_df ["Chance of Winning"]], index = predict_df.index)
    return predict_df

### Base Model Best Picture

In [955]:
diff_class_ml(X_train_best_picture, X_test_best_picture, y_train_best_picture, y_test_best_picture)

,Train error,Validation error,Time in seconds
dummy,0.025,0.051,0.0215
decision tree,0.000,0.034,0.0180
kNN,0.010,0.021,0.0578
logistic regression,0.014,0.021,0.0195
random forest,0.001,0.021,0.0348
xgboost,0.001,0.026,0.1025
lgbm,0.000,0.021,0.0862


In [981]:
#winner was Green Book 
predict_picture_2018 = oscar_predict(X_train_best_picture, y_train_best_picture, best_picture_2018_identifiable, LogisticRegression())
predict_picture_2018

,Classifier,Probability of it Winning
A Star Is Born,No,0.0219739
Bohemian Rhapsody,No,0.0166169
Black Panther,No,0.0146598
The Favourite,No,0.381483
Roma,No,0.354786
Vice,No,0.17434
Green Book,No,0.018428
BlacKkKlansman,No,0.131681


In [982]:
normalized_winner(predict_picture_2018)

,Chance of Winning
A Star Is Born,1.97%
Bohemian Rhapsody,1.49%
Black Panther,1.32%
The Favourite,34.25%
Roma,31.85%
Vice,15.65%
Green Book,1.65%
BlacKkKlansman,11.82%


### Base Model Best Actor

In [958]:
diff_class_ml(X_train_best_actor, X_test_best_actor, y_train_best_actor, y_test_best_actor)

,Train error,Validation error,Time in seconds
dummy,0.027,0.034,0.0204
decision tree,0.000,0.021,0.0183
kNN,0.015,0.021,0.0575
logistic regression,0.014,0.021,0.0190
random forest,0.003,0.021,0.0341
xgboost,0.001,0.021,0.1016
lgbm,0.000,0.021,0.0821


In [991]:
#winner was Bohemian Rhapsody
predict_actor_2018 = oscar_predict(X_train_best_actor, y_train_best_actor, best_actor_2018_identifiable, XGBClassifier())
predict_actor_2018

,Classifier,Probability of it Winning
A Star Is Born,No,0.150907
Bohemian Rhapsody,No,0.0744267
Vice,No,0.300222
At Eternity's Gate,No,0.0452627
Green Book,No,0.212791


In [992]:
normalized_winner(predict_actor_2018)

,Chance of Winning
A Star Is Born,19.26%
Bohemian Rhapsody,9.50%
Vice,38.31%
At Eternity's Gate,5.78%
Green Book,27.16%


### Base Model Best Actress

In [961]:
diff_class_ml(X_train_best_actress, X_test_best_actress, y_train_best_actress, y_test_best_actress)

,Train error,Validation error,Time in seconds
dummy,0.029,0.038,0.0201
decision tree,0.000,0.013,0.0170
kNN,0.015,0.009,0.0575
logistic regression,0.015,0.009,0.0210
random forest,0.000,0.009,0.0328
xgboost,0.000,0.017,0.1158
lgbm,0.000,0.017,0.0848


In [962]:
#winner was The Favourite
predict_actress_2018 = oscar_predict(X_train_best_actress, y_train_best_actress, best_actress_2018_identifiable, XGBClassifier())
predict_actress_2018

,Classifier,Probability of it Winning
A Star Is Born,No,0.156774
The Wife,No,0.0670355
Can You Ever Forgive Me?,No,0.0158528
The Favourite,Yes,0.732928
Roma,Yes,0.782291


In [963]:
normalized_winner(predict_actress_2018)

,Chance of Winning
A Star Is Born,8.93%
The Wife,3.82%
Can You Ever Forgive Me?,0.90%
The Favourite,41.77%
Roma,44.58%


### Base Model Best Adapted Screenplay 

In [964]:
diff_class_ml(X_train_best_ada_screen, X_test_best_ada_screen, y_train_best_ada_screen, y_test_best_ada_screen)

,Train error,Validation error,Time in seconds
dummy,0.034,0.021,0.0224
decision tree,0.000,0.013,0.0208
kNN,0.020,0.009,0.0588
logistic regression,0.017,0.009,0.0202
random forest,0.002,0.004,0.0347
xgboost,0.000,0.009,0.1023
lgbm,0.000,0.009,0.0840


In [965]:
#winner BlacKkKlansman
predict_ada_screen_2018 = oscar_predict(X_train_best_ada_screen, y_train_best_ada_screen, best_adascreen_2018_identifiable, LGBMClassifier())
predict_ada_screen_2018

,Classifier,Probability of it Winning
A Star Is Born,No,0.00047105
Can You Ever Forgive Me?,No,1.6e-06
The Ballad of Buster Scruggs,No,1.457e-05
If Beale Street Could Talk,No,2.33e-06
BlacKkKlansman,No,0.0500786


In [966]:
normalized_winner(predict_ada_screen_2018)

,Chance of Winning
A Star Is Born,0.93%
Can You Ever Forgive Me?,0.00%
The Ballad of Buster Scruggs,0.03%
If Beale Street Could Talk,0.00%
BlacKkKlansman,99.03%


### Base Model Best Director

In [967]:
diff_class_ml(X_train_best_director, X_test_best_director, y_train_best_director, y_test_best_director)

,Train error,Validation error,Time in seconds
dummy,0.033,0.043,0.0228
decision tree,0.000,0.026,0.0183
kNN,0.014,0.030,0.0580
logistic regression,0.015,0.021,0.0190
random forest,0.002,0.034,0.0346
xgboost,0.000,0.038,0.0949
lgbm,0.000,0.034,0.0850


In [968]:
#Winner Roma
predict_director_2018 = oscar_predict(X_train_best_director, y_train_best_director, best_director_2018_identifiable, LGBMClassifier())
predict_director_2018

,Classifier,Probability of it Winning
The Favourite,No,9.06e-05
Roma,No,0.0004692
Vice,No,4.516e-05
Cold War,No,0.00013241
BlacKkKlansman,No,2.441e-05


In [969]:
normalized_winner(predict_director_2018)

,Chance of Winning
The Favourite,11.89%
Roma,61.59%
Vice,5.93%
Cold War,17.38%
BlacKkKlansman,3.20%


### Base Model Best Original Screenplay

In [970]:
diff_class_ml(X_train_best_ori_screen, X_test_best_ori_screen, y_train_best_ori_screen, y_test_best_ori_screen)

,Train error,Validation error,Time in seconds
dummy,0.025,0.034,0.0206
decision tree,0.000,0.021,0.0178
kNN,0.010,0.021,0.0597
logistic regression,0.013,0.021,0.0199
random forest,0.001,0.021,0.0318
xgboost,0.000,0.021,0.0969
lgbm,0.000,0.026,0.0779


In [971]:
#Winner Green Boook
predict_ori_screen_2018 = oscar_predict(X_train_best_ori_screen, y_train_best_ori_screen, best_oriscreen_2018_identifiable, XGBClassifier())
predict_ori_screen_2018

,Classifier,Probability of it Winning
The Favourite,No,0.118976
First Reformed,No,0.0148793
Roma,Yes,0.539255
Vice,No,0.206591
Green Book,No,0.0075816


In [972]:
normalized_winner(predict_ori_screen_2018)

,Chance of Winning
The Favourite,13.41%
First Reformed,1.68%
Roma,60.78%
Vice,23.28%
Green Book,0.85%


### Base Model Best Supporting Actor

In [973]:
diff_class_ml(X_train_best_supporting_actor, X_test_best_supporting_actor, y_train_best_supporting_actor, y_test_best_supporting_actor)

,Train error,Validation error,Time in seconds
dummy,0.036,0.026,0.0197
decision tree,0.000,0.026,0.0191
kNN,0.017,0.009,0.0572
logistic regression,0.017,0.013,0.0205
random forest,0.005,0.009,0.0340
xgboost,0.005,0.009,0.1066
lgbm,0.000,0.009,0.0871


In [974]:
#Winner Green Book
predict_supporting_actor_2018 = oscar_predict(X_train_best_supporting_actor, y_train_best_supporting_actor, best_supporting_actor_2018_identifiable, LogisticRegression())
predict_supporting_actor_2018

,Classifier,Probability of it Winning
A Star Is Born,No,0.17014
Can You Ever Forgive Me?,No,0.025909
Vice,No,0.0493791
Green Book,No,0.0456097
BlacKkKlansman,No,0.0555738


In [975]:
normalized_winner(predict_supporting_actor_2018)

,Chance of Winning
A Star Is Born,49.09%
Can You Ever Forgive Me?,7.47%
Vice,14.25%
Green Book,13.16%
BlacKkKlansman,16.03%


### Base Model Best Supporting Actress

In [976]:
diff_class_ml(X_train_best_supporting_actress, X_test_best_supporting_actress, y_train_best_supporting_actress, y_test_best_supporting_actress)

,Train error,Validation error,Time in seconds
dummy,0.023,0.047,0.0201
decision tree,0.000,0.034,0.0186
kNN,0.010,0.030,0.0556
logistic regression,0.013,0.026,0.0195
random forest,0.000,0.026,0.0346
xgboost,0.001,0.034,0.0960
lgbm,0.000,0.038,0.0805


In [977]:
#Winner If Beale Street Could Talk
predict_supporting_actress_2018 = oscar_predict(X_train_best_supporting_actress, y_train_best_supporting_actress, best_supporting_actress_2018_identifiable, XGBClassifier())
predict_supporting_actress_2018

,Classifier,Probability of it Winning
The Favourite,No,0.0375324
Roma,No,0.165967
Vice,No,0.178952
If Beale Street Could Talk,No,0.0317961


In [978]:
normalized_winner(predict_supporting_actress_2018)

,Chance of Winning
The Favourite,9.06%
Roma,40.06%
Vice,43.20%
If Beale Street Could Talk,7.68%
